# Inteligência na Web e Big Data

## Rodrigo Dias Castelhano RA 21201820796

### Prof Fabrício Olivetti de França 
### Prof Thiago Ferreira Covões



# Criação de Grafos com a bilbioteca GraphX do Spark 


In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
import os
jars    = ["jars/graphframes-0.5.0-spark2.1-s_2.11.jar", "jars/scala-logging-api_2.11-2.1.2.jar", "jars/scala-logging-slf4j_2.11-2.1.2.jar"]
pyfiles = ["pyfiles/graphframes.zip"]
os.environ["PYSPARK_SUBMIT_ARGS"] = " --jars "+ ",".join(jars)  +" --py-files "+ ",".join(pyfiles)  +"  pyspark-shell"

In [3]:
import numpy as np
import pandas as pd
import pyspark
import findspark
from IPython.core.display import HTML
findspark.init()

In [4]:
import json, random
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.types import StructType,StructField,ArrayType

In [5]:
# Lendo a primeira base de dados referente aos usuarios e seus ids

usuarios = spark.read.format("csv").option("header", "true").load("dataset/User.csv").withColumnRenamed("ID", "id")
usuarios.show(5)        

+----+------+
|  id|  NAME|
+----+------+
|1090|Jessie|
|1159|Melvin|
|1436| Hosea|
|1512| Ernst|
|1572|Finley|
+----+------+
only showing top 5 rows



In [6]:
# Lendo a segunda base de dados referente as conexões, temos que trocar o nome das colunas para src e dst

conexoes = spark.read.format("csv").option("header", "true").load("dataset/UserGraph.csv")\
                .withColumnRenamed("USER_1", "src").withColumnRenamed("USER_2", "dst")            
conexoes.show(5)

+----+----+
| src| dst|
+----+----+
|1090|5309|
|1090|3547|
|1090|5166|
|1090|5307|
|1159|1807|
+----+----+
only showing top 5 rows



### Agora podemos criar o grafo a partir do Graph 

In [7]:
from graphframes import *

In [8]:
rede = GraphFrame(usuarios, conexoes) # Carregando para o GraphFrame
print("Total de vertices que são os usuários da rede : %d" % rede.vertices.count())
print('')
print("Total de arestas que são as conexões: %d" % rede.edges.count())
print('')
rede.cache()

Total de vertices que são os usuários da rede : 6486

Total de arestas que são as conexões: 336534



GraphFrame(v:[id: string, NAME: string], e:[src: string, dst: string])

### Agora, por sugestão do professor Fabrício, vou aplicar o algoritmo PagerRank  que tem o objetivo de verificar a relevância de um determinado usuário da rede social. Além dele considerar o número de conexões ele considera também a qualidade (importância) destas relações.  

In [9]:
# Levou 12 minutos para rodar o pagerRank apenas com maxIter=2

results = rede.pageRank(resetProbability=0.15, maxIter=2)

In [10]:
pangerank=results.vertices.select("id", "pagerank","NAME").orderBy("pagerank", ascending=False)
pangerank.show(10)

+----+------------------+---------+
|  id|          pagerank|     NAME|
+----+------------------+---------+
|5306| 33.55305599557101|     Arch|
| 859| 30.20265372649813|   Hallie|
|2664|24.096912979345365|     Edna|
|6306|23.810852330210796| Napoleon|
|5716|21.357154640932315|   Dalton|
|3805|21.127835413682064|Arlington|
|2557| 20.32372457710196|    Giles|
|1602| 19.25657796189257|   Bryant|
|2548| 18.65036287957995|     Dora|
|5736| 18.38009944372988|    Lemon|
+----+------------------+---------+
only showing top 10 rows



### O comando inDegress retorna a quantidade de conexões entre os vértices, portanto vou utilizar este comando para verificar qual o usuário com maior quantidade de conexões nesta simulação de rede social. A seguir vamos listar em ordem decrescente de conexões.
### Podemos então comparar com o resultado obtido para o algoritmo PageRank e mostrar que o nível de relevância não depende apenas da quantidade de conexões mas sim da qualidade destas conexões.

In [11]:
def mais_amigos(rede, q):
    '''
    Vai retornar uma lista com quem tem mais amigos.
    
    '''
    rede_indegrees = rede.inDegrees
        
    return rede.vertices.join(rede_indegrees, "id").orderBy("inDegree", ascending=False).limit(q)

In [12]:
ver_amigos = mais_amigos(rede, rede.vertices.count())
ver_amigos.show(10)

+----+---------+--------+
|  id|     NAME|inDegree|
+----+---------+--------+
| 859|   Hallie|    1933|
|5306|     Arch|    1741|
|2664|     Edna|    1528|
|5716|   Dalton|    1426|
|6306| Napoleon|    1394|
|3805|Arlington|    1386|
|2557|    Giles|    1371|
|4898| Gottlieb|    1345|
|5736|    Lemon|    1289|
| 403|    Alvah|    1280|
+----+---------+--------+
only showing top 10 rows



### Agora vou criar um pequeno DataFrame com nomes de pessoas que tenham cometido suicídio. Obviamente que a pretenção é apenas ilustrar e exemplificar o uso da bibliotéca e portanto não consideramos o efeito prático desta aplicação. 

### Neste caso utilizo apenas o nome da vítima para fazer a busca na rede social o que pode trazer homonimos. Em uma aplicação real a busca deveria ser melhor elaborada.

In [135]:
suicidio = sc.parallelize([('Linn','sim'),('Rodrigo','sim'),('Albert','sim'),('Cato','sim'),('Jonas','sim')
                           ,('Caique','sim'),('Emit','sim'),('João','sim'),('Roy','sim'),( 'Hallie','sim')])
Tabela_suicidio = spark.createDataFrame(suicidio,['NAME','Suicidio'])
Tabela_suicidio.show()

+-------+--------+
|   NAME|Suicidio|
+-------+--------+
|   Linn|     sim|
|Rodrigo|     sim|
| Albert|     sim|
|   Cato|     sim|
|  Jonas|     sim|
| Caique|     sim|
|   Emit|     sim|
|   João|     sim|
|    Roy|     sim|
| Hallie|     sim|
+-------+--------+



### Agora será realizado a junção dos usuários (vertices) da rede social com a pequena base de vítimas de suicídio, trazendo as linhas onde coincidiu o nome da vítima com o usuário da rede social.

In [137]:
rede_suicidio = usuarios.join(Tabela_suicidio, usuarios.NAME == Tabela_suicidio.NAME)
rede_suicidio.show()

+----+------+------+--------+
|  id|  NAME|  NAME|Suicidio|
+----+------+------+--------+
|2904|  Linn|  Linn|     sim|
|2015|Albert|Albert|     sim|
|5015|Albert|Albert|     sim|
|1015|Albert|Albert|     sim|
|6015|Albert|Albert|     sim|
|4015|Albert|Albert|     sim|
|  15|Albert|Albert|     sim|
|3015|Albert|Albert|     sim|
| 829|  Cato|  Cato|     sim|
|6398| Jonas| Jonas|     sim|
|5369| Jonas| Jonas|     sim|
| 322| Jonas| Jonas|     sim|
|4294| Jonas| Jonas|     sim|
|1326| Jonas| Jonas|     sim|
|3361| Jonas| Jonas|     sim|
|2348| Jonas| Jonas|     sim|
|1910|  Emit|  Emit|     sim|
|4894|  Emit|  Emit|     sim|
|  36|   Roy|   Roy|     sim|
|4025|   Roy|   Roy|     sim|
+----+------+------+--------+
only showing top 20 rows



### A função find utiliza-se de modificadores para mapear a busca em Gráfos. Para exibir os usuários conectados a determinada pessoa vou utilizar a função find da seguinte forma:
### (a) - [e] -> (b) , isso quer dizer que um vertice "a" deve estar ligado a aresta "e" a um outro vértice "b". Vamos fazer isso com o usuário Hallie que é quem mais tem conexões e também temos uma vítima de mesmo nome na lista de suicidas. 



In [139]:
def usuarios_conectados(rede, usuarios):
    '''
    Retorna os usuarios conectados
    '''
    return rede.find("(a)-[e]->(b)").filter("b.id = %d" % usuarios).select("a.id", "a.NAME")

l_usuarios = usuarios_conectados(rede,  859)

print('')
print("Total de usuários conectados a Hallie: ", l_usuarios.count())
print('')
l_usuarios.show(5)


Total de usuários conectados a Hallie:  1933

+----+-------+
|  id|   NAME|
+----+-------+
|1572| Finley|
|2069|Patrick|
|2904|   Linn|
|3210|Preston|
|3606|Colonel|
+----+-------+
only showing top 5 rows



### Agora é possível fazer uma nova junção entre as conexões (amigos) de Hallie que teoricamente cometeu suicídio e a Tabela com as vítimas, com isso podemos verificar quais os amigos de Hallie tiveram a mesma atitude que ele e assim verificar algumas semelhanças em suas paginas da rede. Exemplo profissão, gosto pessoal, endereço .... 

In [110]:
suicida = Tabela_suicidio.join(l_usuarios, Tabela_suicidio.NAME == l_usuarios.NAME)
print('')
print('Os amigos de Hallie que também se suicidaram foram : ', suicida.count())
print('')
suicida.show()


Os amigos de Hallie que também se suicidaram foram :  8

+------+--------+----+------+
|  NAME|Suicidio|  id|  NAME|
+------+--------+----+------+
|   Roy|     sim|6019|   Roy|
|Albert|     sim|3015|Albert|
|Albert|     sim|4015|Albert|
|  Cato|     sim| 829|  Cato|
| Jonas|     sim|2348| Jonas|
|  Emit|     sim|4894|  Emit|
|  Emit|     sim|1910|  Emit|
|  Linn|     sim|2904|  Linn|
+------+--------+----+------+



### Vou utilizar os sub-grafos para criar um subgrupo que pode trazer usuários que tem conexões formadas entre dois usuários especificos. Então vamos trazer todos os usuários que tem conexões entre os dois de nossos usuários hipotéticos que cometeram suicídio.


In [17]:
def subgrafo(rede, usuario1_id, usuario2_id):
    '''
   Retorna subgrafo entre os usuarios 1 e 2
    '''
    rede_user  = rede.find("(a)-[e]->(b); (c)-[e2]->(b)").filter("a.id = {} and c.id = {}".format(usuario1_id, usuario2_id))

    rede_vert  = rede_user.select("a.id", "a.NAME").unionAll(rede_user.select("b.id", "b.NAME"))\
                .unionAll(rede_user.select("c.id", "c.NAME")).distinct()

    rede_edges = rede_user.select("e.src", "e.dst").unionAll(rede_user.select("e2.src", "e2.dst")).distinct()

    rede_sub = GraphFrame(rede_vert, rede_edges)

    return rede_sub

In [18]:
usuario1_id = 6019 # Winnie
usuario2_id = 4894 # Finley

# SubGraph with Winnie connections between Finley
rede_users = subgrafo(rede, usuario1_id, usuario2_id)

In [19]:
rede_users.vertices.show(200)

+----+---------+
|  id|     NAME|
+----+---------+
|3805|Arlington|
|5306|     Arch|
|4415| Clarance|
|2650|      Zeb|
|5716|   Dalton|
|6019|      Roy|
|1490| Frederic|
|4366|      Ned|
|6306| Napoleon|
|5736|    Lemon|
|2557|    Giles|
| 859|   Hallie|
|4894|     Emit|
|1347|    Berry|
+----+---------+

